# Hash e Cadeia de Custódia — Laboratório Didático

Este notebook demonstra, passo a passo, como:
1. Calcular **hashes criptográficos** (SHA-256 por padrão) de arquivos.
2. **Registrar** hash + metadados em um **log de custódia** (CSV).
3. Simular **alteração** de arquivo e **detectar** a mudança pelo hash.
4. Fazer **verificação automática** de integridade.

> **Contexto:** Em segurança pública e forense digital, o hash funciona como a **impressão digital** de um arquivo.  
> Qualquer alteração (mesmo 1 byte) muda o hash. Por isso, registramos hash e metadados no momento da aquisição.

In [ ]:
import hashlib
import csv
import os
from datetime import datetime

print("OK: bibliotecas padrão carregadas")

### 1) Funções utilitárias — Cálculo de Hash e Registro de Custódia

In [ ]:
def calcular_hash(caminho_arquivo: str, algoritmo: str = "sha256") -> str:
    "Calcula o hash (SHA-256 por padrão) de um arquivo de forma incremental."
    h = hashlib.new(algoritmo)
    with open(caminho_arquivo, "rb") as f:
        for bloco in iter(lambda: f.read(1024 * 1024), b""):  # 1MB por bloco
            h.update(bloco)
    return h.hexdigest()

def registrar_custodia(csv_caminho: str, arquivo: str, hash_hex: str, responsavel: str, observacao: str = "") -> None:
    "Acrescenta uma linha ao registro de custódia (CSV). Cria cabeçalho se necessário." 
    campos = ["data_iso", "arquivo", "hash_sha256", "responsavel", "observacao"]
    registro = {
        "data_iso": datetime.now().isoformat(),
        "arquivo": arquivo,
        "hash_sha256": hash_hex,
        "responsavel": responsavel,
        "observacao": observacao,
    }
    novo_arquivo = not os.path.exists(csv_caminho)
    with open(csv_caminho, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=campos)
        if novo_arquivo:
            writer.writeheader()
        writer.writerow(registro)

print("OK: funções utilitárias definidas")

### 2) Criar um arquivo de exemplo e calcular o hash

Vamos criar `exemplo.txt` e calcular seu **SHA-256**. Em seguida, registraremos no CSV `custodia.csv`.

In [ ]:
# Criar arquivo de exemplo (conteúdo simples)
arquivo = "exemplo.txt"
with open(arquivo, "w", encoding="utf-8") as f:
    f.write("Relato inicial — ocorrência #12345\n")
    f.write("Data/Hora: 2025-10-24T09:00:00\n")
    f.write("Observações: conteúdo didático para teste de hash.\n")

# Calcular hash SHA-256
hash_inicial = calcular_hash(arquivo, "sha256")
print("Arquivo:", arquivo)
print("SHA-256:", hash_inicial)

### Registrar no log de custódia
Gravamos data/hora, caminho do arquivo, hash e responsável.

In [ ]:
csv_log = "custodia.csv"
registrar_custodia(csv_log, arquivo, hash_inicial, responsavel="aluno_exemplo", observacao="Coleta inicial")
print(f"Registro de custódia atualizado: {csv_log}")
# Mostrar primeiras linhas do CSV
with open(csv_log, "r", encoding="utf-8") as f:
    print("--- custodia.csv ---")
    print(f.read())

### 3) Simular alteração do arquivo (tampering) e recalcular hash

Vamos **alterar 1 linha** do arquivo e ver que o hash muda completamente.

In [ ]:
# Alterar 1 byte/linha no arquivo (simulação de modificação acidental/indevida)
with open(arquivo, "a", encoding="utf-8") as f:
    f.write("Linha adicionada posteriormente (alteração).\n")

hash_modificado = calcular_hash(arquivo, "sha256")
print("SHA-256 (após alteração):", hash_modificado)

# Registrar nova entrada no log de custódia
registrar_custodia(csv_log, arquivo, hash_modificado, responsavel="aluno_exemplo", observacao="Arquivo alterado para demonstração")

with open(csv_log, "r", encoding="utf-8") as f:
    print("--- custodia.csv (atualizado) ---")
    print(f.read())

## 4) Verificação automática de integridade

Função para comparar o hash atual com o hash esperado (registrado).  
Útil para **auditorias** e **pipelines** de ingestão.

In [ ]:
def verificar_integridade(caminho_arquivo: str, hash_esperado: str, algoritmo: str = "sha256") -> bool:
    hash_atual = calcular_hash(caminho_arquivo, algoritmo)
    return hash_atual == hash_esperado

ok = verificar_integridade(arquivo, hash_inicial)
print("Integridade OK (comparando com hash inicial)?", ok)

### 5) Boas práticas (anotações rápidas)

- **Calcule o hash imediatamente após a coleta** (imagem forense, download, exportação).  
- Use algoritmos modernos: **SHA-256** (ou superiores) em vez de MD5.  
- **Registre metadados**: data/hora, operador, origem, ferramenta, local.  
- **Não trabalhe no arquivo original** — sempre em cópias de análise.  
- **Versione o log de custódia** e proteja o diretório de evidências.  
- Em relatórios, **inclua o hash** ao referenciar qualquer arquivo.

> Dica: você pode adaptar este notebook para validar PDFs, imagens forenses e dumps de banco de dados—basta apontar `calcular_hash()` para o arquivo desejado.
